In [ ]:
# conectar seu drive caso você queira salvar ou carregar algo de lá

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Funçoes e importações

In [ ]:
from IPython.display import clear_output
import sys
import ipywidgets as widgets
from IPython.display import display, HTML
import os
from IPython.display import clear_output
from google.colab import userdata
!pip install basedosdados==2.0.0b16
clear_output()
!pip install gql
!pip install requests-toolbelt
!git clone --branch colab-scripts https://github.com/basedosdados/databasers-utils.git
sys.path.insert(0, '/content/databasers-utils/colab-scripts')
clear_output()

from gql import gql

In [ ]:
from client_graphql import MakeClient
import pandas as pd
from datetime import datetime
from typing import NamedTuple

def get_tables_null() -> dict:

  query = """
        query{
          allTable(updates_Id_Isnull: true){
            edges{
              node{
                id
                name
                slug
                dataset{
                  id
                  name
                  slug
                }
                cloudTables{
                  edges{
                    node{
                      gcpTableId
                      gcpDatasetId
                    }
                  }
                }
              }
            }
          }
        }
    """

  respose = client.query.execute(gql(query))
  return respose


def make_row(short: dict) -> dict:

  row = {}

  row["table_id"] = short["node"]["id"]
  row["table_name"] = short["node"]["name"]
  row["table_slug"] = short["node"]["slug"]
  row["dataset_id"] = short["node"]["dataset"]["id"]
  row["dataset_name"] = short["node"]["dataset"]["name"]
  row["dataset_slug"] = short["node"]["dataset"]["slug"]

  try:
    row["bq_table"] = short["node"]["cloudTables"]["edges"][0]["node"]["gcpTableId"]
    row["bq_dataset"] = short["node"]["cloudTables"]["edges"][0]["node"]["gcpDatasetId"]
  except IndexError:
    row["bq_table"] = None
    row["bq_dataset"] = None

  return row


def clear_id(id_value: str) -> str:
  return id_value.split(":")[1]


def form_link(base_url: str, table_id: str, dataset_id: str) -> str:
  return base_url.format(table_id=table_id, dataset_id=dataset_id)


def create_generic_update(row: NamedTuple) -> str:

  inputs = {
      "table": row.table_id,
      "latest": row.last_modified_time,
      "entity": "81f0c890-65a6-48a1-9523-af38d3f4af63"
  }

  response = client.query_mutation(mutation_class="Update", input_values=inputs, only_id=True)
  return response


# Constantes

In [ ]:
client = MakeClient()

base_page_link = "https://basedosdados.org/dataset/{dataset_id}?table={table_id}"
base_backend_link_table = "https://backend.basedosdados.org/admin/v1/table/{table_id}"
base_backend_link_dataset = "https://backend.basedosdados.org/admin/v1/dataset/{dataset_id}"
path_output_excel = "/content/tabelas_sem_update.xlsx"
path_base_excel = "/content/drive/MyDrive/DataBase/bd/tabelas_sem_update.xlsx"

# Coleta

## Pega vazios

In [ ]:
response_nulls = get_tables_null()

rows = [make_row(short) for short in response_nulls["allTable"]["edges"]]
df_nulls = pd.DataFrame(rows)
df_nulls.table_id = df_nulls.table_id.apply(clear_id)
df_nulls.dataset_id = df_nulls.dataset_id.apply(clear_id)
df_nulls["page_link"] = [form_link(base_page_link, row.table_id, row.dataset_id) for row in df_nulls.itertuples()]
df_nulls["backend_table_link"] = [form_link(base_backend_link_table, row.table_id, row.dataset_id) for row in df_nulls.itertuples()]
df_nulls["backend_dataset_link"] = [form_link(base_backend_link_dataset, row.table_id, row.dataset_id) for row in df_nulls.itertuples()]

para_coletar = df_nulls[(df_nulls.bq_dataset.notnull()) & (df_nulls.bq_dataset != "br_mc_auxilio_emergencial")]

## Pegar ultima data de materialização bigquery

In [ ]:
# Formar Query para coleta a data de materialização

query = """
tabelas{number} as (
SELECT
dataset_id,
table_id,
TIMESTAMP_MILLIS(creation_time) as last_modified_time
FROM `basedosdados.{bq_dataset}.__TABLES_SUMMARY__`
WHERE table_id = '{bq_table}')
"""


query_done = ", \n".join([query.format(number=n, bq_dataset=row.bq_dataset, bq_table=row.bq_table) for n, row in enumerate(para_coletar.itertuples())])

selects = "union all\n".join([f"select * from tabelas{n}\n" for n in range(len(para_coletar))])

query_full = "with " + query_done + "\n" + selects

# Salvar query caso debug
with open("query_table.txt", "w") as query_w:
  query_w.write(query_full)

In [ ]:
import basedosdados as bd

df_last_modified_time = bd.read_sql(
      query_full,
      billing_project_id="basedosdados-dev"
  )

Downloading: 100%|██████████|


## Juntar ultima data do bigquery com nosso dataframe

In [ ]:
for row in df_last_modified_time.itertuples():
  mask = (para_coletar.bq_dataset == row.dataset_id) & (para_coletar.bq_table == row.table_id)
  para_coletar.loc[mask, "last_modified_time"] = row.last_modified_time

para_coletar.last_modified_time = para_coletar.last_modified_time.apply(lambda date: date.isoformat())
para_coletar["update_id"] = ""

para_coletar.to_excel(path_output_excel, index=False)

# Criar Update

In [ ]:
tabela_sem_update = pd.read_excel(path_base_excel,
                               dtype=str)

select_rows = (tabela_sem_update.update_id.isnull()) & (tabela_sem_update.last_modified_time != "NaT")
para_preencher = tabela_sem_update[select_rows]

for row in para_preencher.itertuples():

  id_update = create_generic_update(row)

  tabela_sem_update.loc[row.Index].update_id = id_update

  tabela_sem_update.to_excel(path_base_excel, index=False)